Установка tensorflow для дальнейшей работы с нейронными сетями

In [ ]:
!pip install -q tensorflow==2.3

Установка albumentations для аугментации изображений

In [ ]:
# аугментация изображений
!pip install albumentations - q

Морфологический анализатор для работы с текстом:

In [ ]:
!pip install pymorphy2
!pip install pymorphy2-dicts

Теперь импортируем необходимые для работы библиотеки

In [ ]:
import seaborn as sns
import random
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import pymorphy2
from nltk.corpus import stopwords
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

from scipy.optimize import minimize

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers.schedules import *
import albumentations as A

# plt
import matplotlib.pyplot as plt
# увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline


Проверим версии используемых библиотек

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

Функция для вычисления метрики MAPE

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

Теперь установим random seed для воспроизводимости:

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

Зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:

In [ ]:
!pip freeze > requirements.txt

# 1. DATA

Посмотрим на типы признаков:

* bodyType - категориальный
* brand - категориальный
* color - категориальный
* description - текстовый
* engineDisplacement - числовой, представленный как текст
* enginePower - числовой, представленный как текст
* fuelType - категориальный
* mileage - числовой
* modelDate - числовой
* model_info - категориальный
* name - категориальный, желательно сократить размерность
* numberOfDoors - категориальный
* price - числовой, целевой
* productionDate - числовой
* sell_id - изображение (файл доступен по адресу, основанному на sell_id)
* vehicleConfiguration - не используется (комбинация других столбцов)
* vehicleTransmission - категориальный
* Владельцы - категориальный
* Владение - числовой, представленный как текст
* ПТС - категориальный
* Привод - категориальный
* Руль - категориальный

Укажем директорию и загрузим наш обучающий и тестовый фреймы:

In [ ]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

Проверим, что у нас находится в обучающем и тестовом фреймах

In [ ]:
train.info()

In [ ]:
train.nunique()

In [ ]:
train.isna().sum()

Присутствуют пропуски, которые в дальнейшем придется обработать.Проверим,что находится в тестовом фрейме:

In [ ]:
test.info()

In [ ]:
test.nunique()

In [ ]:
test.isna().sum()

Здесь пропуски имеются только в столбце "владение".

Почистим дубликаты

In [ ]:
train.drop_duplicates(inplace = True)

In [ ]:
train.info()

Полные дубликаты отсутствуют, потому что sell_id для каждой колонки разный. Посмотрим, есть ли ситуации, в которых совпадает всё, кроме sell_id:

In [ ]:
col_list = list(train.columns.values)
col_list.remove('sell_id')

In [ ]:
train.duplicated(subset=col_list).value_counts()

In [ ]:
train = train[train.duplicated(subset=col_list) == False] 

In [ ]:
train.info()

# 2. Model 0: "Наивная" модель 
Эта модель будет предсказывать среднюю цену по модели и году выпуска. 
C ней будем сравнивать другие модели.



Первым делом проведем разделенение на обучающую и валидационную выборки:

In [ ]:
# split данных
data_train, data_test = train_test_split(
    train, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

Создадим наивную модель и проведем оценку отчности:

In [ ]:
# Наивная модель
predicts = []
for index, row in pd.DataFrame(data_test[['model_info', 'productionDate']]).iterrows():
    query = f"model_info == '{row[0]}' and productionDate == '{row[1]}'"
    predicts.append(data_train.query(query)['price'].median())

# заполним не найденные совпадения
predicts = pd.DataFrame(predicts)
predicts = predicts.fillna(predicts.median())

# округлим
predicts = (predicts // 1000) * 1000

# оцениваем точность
print(
    f"Точность наивной модели по метрике MAPE: {(mape(data_test['price'], predicts.values[:, 0]))*100:0.2f}%")

Мы получили достаточно плохой результат, но теперь появилось понимание - откуда отталкиваться. 

Первым делом проведем EDA и попробуем использовать CatBoost.

# 3. EDA и Feature-Engeneering

Посмотрим, как выглядят распределения числовых признаков:

In [ ]:
# посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
    columns = min(3, len(titles_values_dict))
    rows = (len(titles_values_dict) - 1) // columns + 1
    fig = plt.figure(figsize=(columns * 6, rows * 4))
    for i, (title, values) in enumerate(titles_values_dict.items()):
        hist, bins = np.histogram(values, bins=20)
        ax = fig.add_subplot(rows, columns, i + 1)
        ax.bar(bins[:-1], hist, width=(bins[1] - bins[0]) * 0.7)
        ax.set_title(title)
    plt.show()


visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

Итого:
* CatBoost сможет работать с признаками и в таком виде, но для нейросети нужны нормированные данные.

## 3.1. Предобработка табличных данных

Объединим тестовый и обучающий фреймы в один:

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1  # трейн
test['sample'] = 0  # тест
test['price'] = np.nan # в тесте нет значения price,  заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True)  # объединяем
print(train.shape, test.shape, data.shape)

Теперь приступим к предобработке

### 3.1.1. bodyType

Данный столбец отражает тип кузова автомобиля. Посмотрим на количество вариантов:

In [ ]:
data.bodyType.value_counts()

В датасете есть образцы типа кузова, данных по которым мало. Посмотрим, сможем ли мы сгруппировать эти разделы в более плотные. Стоит убрать количество дверей в виде отдельного наименования.

In [ ]:
data.bodyType.unique()

Проведем обработку данных:

In [ ]:
# Выбираем первое слово для описания типа кузова
data['bodyType'] = data['bodyType'].astype(
    str).apply(lambda x: None if x.strip() == '' else x)
# Понижаем регистр первого слова
data['bodyType'] = data.bodyType.apply(lambda x: x.split(' ')[0].lower())

Посмотрим количество уникальных значений:

In [ ]:
data.bodyType.value_counts()

In [ ]:
data.bodyType.unique()

Проверим распределение значений по типу кузова:

In [ ]:
data.bodyType.value_counts().plot.barh()

В дальнейшем признак отнесем к категориальным.

### 3.1.2. brand

Данный столбец отражает название фирмы-производителя автомобиля. Проверим уникальные значения:

In [ ]:
data.brand.unique()

Уникальных значений мало, можно оставить без изменений. Посмотрим распределение значений:

In [ ]:
data.brand.value_counts().plot.barh()

Отметим, что марки машин представлены более-менее одинаково. Отнесем признак к категориальным.

### 3.1.3. color

Посмотрим количество уникальных значений:

In [ ]:
data.color.value_counts()

Посмотрим распределение значений по цветам:

In [ ]:
data.color.value_counts().plot.barh()

Тут уже все хорошо построено, этот признак можно не дообрабатывать

### 3.1.4. description

Данный столбец содержит комментарий о продаваемом автомобиле. В дальнейшем будет использоваться для NLP. Сейчас выделим на основе этого столбца признак, отражающий длину комментария:

In [ ]:
data['comment_length'] = data.description.apply(lambda x: len(str(x)))

In [ ]:
data.comment_length.hist()

Отнесем к числовым признакам

### 3.1.5. engineDisplacement

В датасете объем двигателя указан в литрах (с точностью до десятых). Таким образом, мы можем преобразовать значение в литрах к значению в кубических сантиметрах, используя следующие функции:

In [ ]:
expr = re.compile('\d.\d')
def LTR_to_cc(expression):
    result = expr.search(expression)
    if result !=None:
        return(round(float(result[0])*1000))
    else:
        return(None)
expr_2 = re.compile('\d+')    
def extract_power(expression):
    result = expr_2.search(expression)
    return(int(result[0]))

Этот признак описывает объем двигателя авто. Посмотрим количество уникальных значений :

In [ ]:
data.engineDisplacement.unique()

Признак отражает литраж двигателя с припиской LTR. Попробуем ее убрать:

In [ ]:
data.engineDisplacement = data.engineDisplacement.apply(lambda x:LTR_to_cc(x))

In [ ]:
data.engineDisplacement.unique()

In [ ]:
data.engineDisplacement.isna().value_counts()

6 значений имеют und ( остаток от undefined LTR). Заполним значением моды:

In [ ]:
data.engineDisplacement = data.engineDisplacement.fillna(data.engineDisplacement.mode()[0])

Проверим результат:

In [ ]:
data.engineDisplacement.unique()

In [ ]:
data.engineDisplacement.isna().value_counts()

Преобразуем в числовой формат и посмотрим распределение:

In [ ]:
data.engineDisplacement = data.engineDisplacement.apply(lambda x: float(x))

In [ ]:
data.engineDisplacement.hist(bins = 30)

Распределение имеет два ярко выраженных пика и тяжелый правый хвост. Логарифмирование в таком случае даст мало преимуществ. (Потом сравним методы, при котором мы относимся к EngineDisplacement как к категориальному признаку)

In [ ]:
data['engineDisplacement_log'] = np.log(data.engineDisplacement)

In [ ]:
data.engineDisplacement_log.hist()

Теперь получили какую-то непонятную бимодальную конструкцию вместо распределения.

### 3.1.6. enginePower

Признак отражает мощность двигателя автомобиля. Посмотрим на уникальные значения:

In [ ]:
data.enginePower.unique()

Избавимся от дополнительной информации в данных:

In [ ]:
# берем 1 символ, если длина строки 5, первые два если 6,в противном случае первые три символа
data['enginePower'] = data['enginePower'].apply(
    lambda x: extract_power(x))

Проверим результат:

In [ ]:
data.enginePower.unique()

Переведем в числовой формат и посмотрим распределение значений:

In [ ]:
data['enginePower'] = data['enginePower'].apply(lambda x: int(x))
data.enginePower.hist(bins = 30)

Распределение близко к нормальному, но возможно необходимо будет логарифмировать для улучшения показателей. Отнесем к числовым признакам

In [ ]:
data['enginePower_log'] = np.log(data.enginePower)

In [ ]:
data.enginePower_log.hist(bins = 30)

### 3.1.7. fuelType

Данный признак содержит данные о типе топлива, используемого в автомобиле. Посмотрим на уникальные значения:


In [ ]:
data.fuelType.value_counts()

Посмотрим распределение признака:

In [ ]:
data.fuelType.hist()

Как видно из графика, подавляющее большинство автомобилей используют бензиновое топливо. Отнесем признак к категориальным.

### 3.1.8. mileage

Признак содержит данные о пробеге продаваемого авто. Проверим данные:

In [ ]:
data.mileage.unique()

Данные чистые. Посмотрим на распределение признака:

In [ ]:
data.mileage.hist(figsize=(8, 5), bins=100)

Проверим на выбросы:

In [ ]:
IQR = data['mileage'].quantile(0.75) - data['mileage'].quantile(0.25)
perc25 = data['mileage'].quantile(0.25)  # 25-й перцентиль
perc75 = data['mileage'].quantile(0.75)  # 75-й перцентиль

print(
    '25-й перцентиль: {},'.format(perc25),
    '75-й перцентиль: {},'.format(perc75),
    "IQR: {}, ".format(IQR),
    "Границы выбросов: [{f}, {l}].".format(f=0,
                                           l=perc75 + 1.5*IQR))

Посмотрим, сколько значений превышает границу:

In [ ]:
data.mileage[data.mileage > 328841].count()

Около 2% из всего датасета, что, в общем-то, может соответствовать действительности. Само распределение имеет тяжелый правый хвост - попробуем логарифмирование

Отнесем признак к числовым и в дальнейшем используем нормализацию.

In [ ]:
data['mileage_log'] = np.log(data.mileage + 1)

In [ ]:
data.mileage_log.hist(figsize=(8, 5), bins=100)

Теперь получили такой же левый хвост, но само распределение стало чуть ближе (хотя бы визуально) к нормальному

### 3.1.9. modelDate

Данный признак содержит информацию о годе производства модели авто. Просмотрим уникальные значения:

In [ ]:
data.modelDate.unique()

Теперь построим график распределения для наглядности:

In [ ]:
data.modelDate.hist()

Автомобили, представленные в датасете, производились в промежутке с 1970 по 2019 год. Создадим новый признак, отражающий возраст модели:

In [ ]:
data['model_time'] = datetime.now().year - data.modelDate

Это числовой признак в явном виде, таким его и оставим

In [ ]:
data['model_time_log'] = np.log(data.model_time + 1)

### 3.1.10 model_info

Данный столбец отражает информацию о модели продаваемого автомобиля. Посмотрим количество уникальных значений:

In [ ]:
data.model_info.unique()

Мы имеем одну строчку без указания модели. Посмотрим, что это за строка:

In [ ]:
data.loc[data['model_info'] == 'None']

Видим, что это Mercedes-седан. Посмотрим, вдруг возможно у нас есть похожие по характеристикам автомобили:

In [ ]:
data[(data['brand'] == 'MERCEDES') & (data['engineDisplacement'] == 4000.0) & (data['enginePower'] == 510)
     & (data['fuelType'] == 'бензин') & (data['Привод'] == 'задний') & (data['bodyType'] == 'седан')].model_info

In [ ]:
data['model_info'][2803] = 'C_KLASSE_AMG'

In [ ]:
data.loc[data['model_info'] == 'None']

### 3.1.11. name

Столбец дублирует информацию из других столбцов. Единственное, что можно из него выделить - это наличие xDrive в автомобиле. Создадим новый признак:

In [ ]:
data['xDrive'] = data['name'].apply(lambda x: 1 if 'xDrive' in x else 0)

Посмотрим как распределен признак:

In [ ]:
sns.countplot(x='xDrive', data=data)

Как мы видим, большая часть представленных автомобилей не имеет xDrive. Отнесем к категориальным признакам.

### 3.1.12. numberOfDoors

Признак содержит информацию о количестве дверей авто. Посмотрим на уникальные значения:

In [ ]:
data.numberOfDoors.unique()

Как видно, никаких значений, вызывающих сомнения, нет. Проверим распределение признака:

In [ ]:
sns.countplot(x='numberOfDoors', data=data)

Большая часть авто имеют 4 или 5 дверей. Отнесем к категориальным признакам.

### 3.1.13. productionDate

Признак содержит информацию о годе производства авто. Проверим корреляцию между productionDate и modelDate:

In [ ]:
data[['modelDate', 'productionDate']].corr()

In [ ]:
data.productionDate.hist()

In [ ]:
data['productionDate_log'] = np.log(data.productionDate + 1)

Корреляция очень высокая, значит один признак почти дублирует другой, а это может негативно сказаться на обучении. Во избежание этого, оставим только один из признаков, то есть modelDate, который мы добавили выше.

### 3.1.14. sell_id

Этот признак нам в дальнейшем пригодится для того, чтобы находить изображения по идентификатору объявления. Для табличного обучения информативности не имеет.

### 3.1.15. vehicleConfiguration

Столбец содержит информацию из других столбцов. Информативности не несет

### 3.1.16. vehicleTransmission

Признак содержит информацию о типе коробки передач, используемой в транспортном средстве. Проверим уникальные значения:

In [ ]:
data.vehicleTransmission.unique()

Посмотрим распределение признака:

In [ ]:
sns.countplot(x='vehicleTransmission', data=data)

Подавляющее большинство автомобилей имеют автоматическую коробку передач. Отнесем к категориальным признакам.

### 3.1.17. Владельцы

Признак содержит информацию о количестве владельцев авто. Посмотрим пропуски:

In [ ]:
data.Владельцы.isna().sum()

Всего один пропуск. Посмотрим, что это за строка:

In [ ]:
data[data['Владельцы'].isnull()]

Посмотрим описание, возможно там указано, сколько было владельцев:

In [ ]:
data.iloc[6665]['description']

владелец как минимум один у неё был. Поэтому поставим 1.

In [ ]:
data.Владельцы = data.Владельцы.apply(
    lambda x: '1' if pd.isna(x) else x)

Посмотрим результат:

In [ ]:
data.Владельцы.isna().sum()

Пропущенное значение заполнилось значением моды для этого столбца. Теперь оставим только числовые значения и приведем к числовому виду столбец:

In [ ]:
data['Владельцы'] = data['Владельцы'].apply(
    lambda x: int(x[0])).astype('int32')

Посмотрим распределение:

In [ ]:
data.Владельцы.hist()

Автомобили с 3 владельцами немного преобладают над всеми остальными, но не сильно. Отнесем признак к категориальным.

### 3.1.18. Владение

Данный столбец отражает,сколько времени продавец владеет автомобилем. Посмотрим, сколько пропусков в данном столбце:

In [ ]:
data.Владение.isna().sum()

Посмотрим соотношение пропусков ко всему дата-сету:

In [ ]:
data.Владение.isna().sum()/len(data.Владение)*100

Пропусков почти 65% датасета. Заполнение их каким-либо одним значением сыграет только плохую роль. Лучше данный столбец вообще не включать в обучение.

### 3.1.19. ПТС

Данный признак содержит информацию о ПТС автомобиля. Проверим уникальные значения:

In [ ]:
data.ПТС.unique()

Выведем график распределения признака:

In [ ]:
data.ПТС.hist()

Большая часть продавцов имеет оригинал ПТС. Пропусков не наблюдается. Отнесем признак к категориальным.

### 3.1.20. Привод

Признак содержит информацию о том, какой привод у транспортного средства. Посмотрим график распределения признака:

In [ ]:
sns.countplot(x='Привод', data=data)

Большинство автомобилей имеют полный привод. Пропусков нет, отнесем признак к категориальным.

### 3.1.21. Руль

Признак отражает положение руля в автомобиле. Посмотрим распределение признака:

In [ ]:
sns.countplot(x='Руль', data=data)

Почти все автомобили имеют левый руль, пропусков нет. Отнесем признак к категориальным.

### 3.1.22. Целевой признак - price

Искомый признак, отражающий стоимость автомобиля. Присутствует только в тренировочной выборке. Посмотрим, в каком виде представлена цена:

In [ ]:
data.iloc[6665]['price']

Посмотрим график распределения целевого признака:

In [ ]:
plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.title(f"Распределение столбца {'price'}")
sns.distplot((data[data['sample'] == 1]['price']), bins=50)

plt.subplot(1, 2, 2)
sns.boxplot(data['price'])
plt.xlabel('Price')
plt.title(f"Боксплот столбца {'price'}", fontsize=12)
plt.show()
data.price.describe()

Данные сильно смещены влево. Посмотрим, как будет себя весть признак после логарифмирования:

In [ ]:
plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.title(f"Распределение после log.{'price'} ")
sns.distplot(np.log(data[data['sample'] == 1]['price']), bins=50)

plt.subplot(1, 2, 2)
sns.boxplot(np.log(data[data['sample'] == 1]['price']))
plt.xlabel('Price')
plt.title('Боксплот после log.price', fontsize=12)
plt.show()

Распределение стало более центрированным, что хорошо. Выбросы трогать не будем, так как стоимость автомобиля указывает владелец, то есть она может быть довольно (а иногда - необоснованно) высокой.

Теперь посмотрим зависимости некоторых столбцов от целевого:

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter((data.price), data.model_time)

Из распределения видно, что чем более "свежий" автомобиль, тем выше на него цена. Но опять же, некоторые более старые автомобили могут стоить дорого из-за раритетности. Посмотрим также зависимость от привода:

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter((data.price), data.Привод)

Опять же, из распределения хорошо видно, что наиболее высокая цена у полного привода, что логично.

In [ ]:
data.sample()

In [ ]:
data.comment_length.hist()

In [ ]:
data['comment_length_log'] = np.log(data.comment_length + 1)

In [ ]:
data.comment_length_log.hist()

Теперь сформируем перечень категориальных и числовых признаков:

In [ ]:
# используем все текстовые признаки как категориальные без предобработки
cat_features = ['bodyType', 'brand', 'color', 'fuelType', 'model_info',
                'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'xDrive']

# числовые признаки
num_features = ['productionDate_log','mileage_log',
                'engineDisplacement_log', 'enginePower_log', 'comment_length_log', 'productionDate','mileage',
                'engineDisplacement', 'enginePower', 'comment_length']

Необходимо посмотреть корреляционную зависимость наших признаков:

In [ ]:
plt.figure(figsize=(12, 7))
sns.heatmap(data[data['sample'] == 1]
            [num_features + ['price']].corr(), annot=True)

Есть сильная обратная зависимость между признаками model_time и modelDate. Нейронная сеть simple dense NN очень сильно теряет в MAPE, если убрать хотя бы один из признаков. В рамках эксперимента я поменеял modelDate на productionDate. Корреляция упала с -1 до -0.964, что уже дает хоть какой-то результат. При этом catBoost и DNN стали вести себя гораздо лучше (прогресс со 107 места leaderboard'a на 41). В итоге было решено добавить вместо modelDate в числовые фичи productionDate. Ниже представлена их корреляция и изменение столбца num_features.

In [ ]:
# числовые признаки
num_features = ['productionDate_log','mileage_log',
                'engineDisplacement_log', 'enginePower_log', 'comment_length_log','productionDate','mileage',
                'engineDisplacement', 'enginePower', 'comment_length']

Теперь напишем функцию для финального этапа обработки данных:

In [ ]:
def preproc_data(df_input):

    df_output = df_input.copy()

    # Удалим неиспользуемые столбцы
    df_output.drop(['description', 'sell_id', 'vehicleConfiguration',
                    'Владение', 'name', 'modelDate'], axis=1, inplace=True)

    ############################### Нормализация ####################################################################

    scaler = RobustScaler()  
    for column in num_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:, 0]

    #################### Работа с категориальными признаками ############################################################
    # Label Encoding
    for column in cat_features:
        df_output[column], _ = pd.factorize(df_output[column])

    # One-Hot Encoding:
    df_output = pd.get_dummies(
        df_output, columns=cat_features, dummy_na=False)

    return df_output

Применим полученную функцию для предобработки наших данных:

In [ ]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
df_preproc.sample(10)

Теперь, когда мы провели предобработку данных, можно приступать к формированию обучающей и валидационной выборок и исследованию различных моделей 

## 3.2. Split data

Разделим данные обратно на тестовую и обучающую выборки:

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

Посмотрим правильно ли все разделилось:

In [ ]:
test_data.info()

Если посмотреть в самое начало, то мы увидим, что правильно: 1671 строка.

Теперь создадим первую не "наивную" модель на основе CatBoost.

# 4. Model 2: CatBoostRegressor

Разделим наш train на обучающую и валидационную выборки:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

Теперь обучим наш CatBoostRegressor:

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'learning_rate':[1e-3,1e-2,1e-1], 'depth':[6,7,8,9,10]}

In [ ]:
model = CatBoostRegressor(iterations=6000,  # Количество итераций
                          random_seed=RANDOM_SEED,
                          depth = 9, learning_rate = 0.01,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,  # Прерывает выполнение, если нет улучшения 500 итераций
                          # task_type='GPU',
                          )
# обучим модель
scorer = make_scorer(mape)

In [ ]:
#clf = GridSearchCV(model, parameters, scoring=scorer)
#clf.fit(X_train, np.log(y_train),eval_set=(X_test, np.log(y_test)),
#          verbose_eval=1000,
#          use_best_model=True)

In [ ]:

model.fit(X_train, np.log(y_train),
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=1000,
          use_best_model=True,
          # plot=True
          )

Посмотрим, какая у нас получилась метрика:

In [ ]:
test_predict_catboost = np.exp(model.predict(X_test))
print(f"TEST mape: {(mape(y_test, test_predict_catboost))*100:0.2f}%")

 Для CatBoost MAPE получилась выше со столбцами

### Submission

Создадим сабмит:

In [ ]:
sub_predict_catboost = np.exp(model.predict(X_sub))
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('cat_sub.csv', index=False)

По итогам сабмита получили MAPE порядка 11.75%

# 5. Model 3: Табличная нейронная сеть

In [ ]:
#num_features = ['productionDate_log','mileage_log',
#                'engineDisplacement_log', 'enginePower_log', 'comment_length_log']

num_features = ['productionDate_log','mileage_log',
                'engineDisplacement_log', 'enginePower_log', 'comment_length_log','productionDate','mileage',
                'engineDisplacement', 'enginePower', 'comment_length']

In [ ]:
def preproc_data_NN(df_input):
    df_output = df_input.copy()
    # Удалим неиспользуемые столбцы
    df_output.drop(['description', 'sell_id', 'vehicleConfiguration',
                    'Владение', 'name', 'modelDate','productionDate','mileage',
                'engineDisplacement', 'enginePower', 'comment_length'], axis=1, inplace=True)
    ############################### Нормализация ####################################################################
    scaler = RobustScaler()  
    for column in num_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:, 0]
    #################### Работа с категориальными признаками ############################################################
    # Label Encoding
    for column in cat_features:
        df_output[column], _ = pd.factorize(df_output[column])
    # One-Hot Encoding:
    df_output = pd.get_dummies(
        df_output, columns=cat_features, dummy_na=False)
    return df_output

In [ ]:
df_preproc_NN = preproc_data(data)
df_preproc_NN.sample(10)

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc_NN.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc_NN.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, shuffle=True, random_state=RANDOM_SEED)

Проверим наш обучающий дата-сет:

In [ ]:
X_train.head(5)

### 5.1. Simple Dense NN

Создадим простую Dense сеть.

На данном этапе было перепробовано несколько подходов с различным количеством и размером dense-слоев, функциями активации, количеством dropout-слоев. Наиболее удачным оказался вариант с активацией relu всех dense-слоев и kernel-регуляризацией. были исследованы другие способы регуляризации, но результат получался хуже.

Финальный вариант исследуемой сети представлен ниже:

In [ ]:
model = Sequential()
model.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model.add(L.Dropout(0.5))
model.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-5), activation="relu"))
model.add(L.Dropout(0.25))

#Экспериментальная вставка
model.add(L.Dense(32, kernel_regularizer=regularizers.l2(
    l2=1e-5), activation="relu"))
#Экспериментальная вставка

model.add(L.Dense(1, activation="linear"))

Посмотрим структуру модели:

In [ ]:
model.summary()

Укажем оптимизатор с скоростью обучения 0.005 и скомпилируем модель:

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(0.004)
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

Пропишем наши коллбеки:
* earlystop - Для прерывания обучения в случае, когда нет положительного результата на протяжении **patience** эпох.
* checkpoint - Сохраняет лучшие веса модели

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=50, restore_best_weights=True,)
callbacks_list = [checkpoint, earlystop]

### 5.2. Fit

Обучаем нашу простую модель в течение 500 эпох. По факту, до конца это обучение дойдет вряд ли, потому что рано или поздно сработает EarlyStopping, который прервет процесс обучения, если не было положительной динамики 50 эпох. Валидацию проводим на валидационной части нашего дата-сета.

In [ ]:
history = model.fit(X_train, (y_train),
                    batch_size=512,  # размер батча
                    epochs=800,  # количество эпох для обучения
                    validation_data=(X_test, (y_test)),  # данные для валидации
                    callbacks=callbacks_list,  # список  наших коллбэков
                    verbose=1,  # параметр, отвечающий за выведение прогресс-бара
                    )

Посмотрим график Loss-функции:

In [ ]:
plt.title('Loss Function')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

Из графика видно, что вначале происходит резкий спад функции, но потом понижение происходит очень медленно, и к ~ 250 эпохе срабатывает прерывание выполнения из-за отсутствия положительного результата.

Сохраним модель и загрузим лучшие веса:

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_1.hdf5')

Проверим исследуемую метрику:

In [ ]:
test_predict_nn1 = model.predict(X_test)
print(f"TEST mape: {(mape(y_test, test_predict_nn1[:,0]))*100:0.2f}%")

Метрика получилась гораздо лучше, чем при использовании CatBoostRegressor'a, значит мы движемся в правильном направлении.

Сделаем сабмит:

In [ ]:
sub_predict_nn1 = model.predict(X_sub)
sample_submission['price'] = sub_predict_nn1[:, 0]
sample_submission.to_csv('nn1_submission.csv', index=False)

На данном этапе результат mape:11.03% на тестовой выборке, что улучшило предыдущий результат на 0.75%.


# 5 3/4. Blending NN + ML

In [ ]:
blend_predict_1 = (test_predict_catboost + test_predict_nn1[:,0]) / 2
print(f"TEST mape: {(mape(y_test, blend_predict_1))*100:0.2f}%")

In [ ]:
blend_sub_predict = (sub_predict_catboost +
                     sub_predict_nn1[:, 0] ) / 2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)

In [ ]:
def mape_function(coef_array,pred_array):
    final_pred = (1-abs(np.sum(coef_array)))*pred_array[0]
    for i in range(N_models-1):
        final_pred += coef_array[i]*pred_array[i+1]
    F = mape(y_test, final_pred)
    return F

In [ ]:
N_models = 2
initial_guess = np.ones(N_models-1)/(N_models)
bnd = ((-1,1),(-1,1))
predictions = [test_predict_catboost,test_predict_nn1[:,0]]
fun = lambda x: mape_function(x,predictions)
res = minimize(fun, initial_guess, bounds = bnd, method='Nelder-Mead', tol=1e-6)
res.x

In [ ]:
final_pred = (1-np.sum(res.x))*predictions[0]
for i in range(N_models-1):
    final_pred += res.x[i]*predictions[i+1]
mape(y_test, final_pred)

In [ ]:
optimized_sub_pred = (1-np.sum(res.x))*sub_predict_catboost
optimized_sub_pred += res.x[0]*sub_predict_nn1[:, 0]
mape(y_test, final_pred)

sample_submission['price'] = optimized_sub_pred
sample_submission.to_csv('blend_submission_optimized.csv', index=False)

# 6. Model 4: NLP + MLP Multi Input NN

Для данной модели мы будем использовать две сети:
* NLP для обработки столбца description
* MLP, созданная ранее, для табличных данных

Для начала посмотрим, как выглядят данные в столбце:

In [ ]:
data.description

Столбец представляет собой комментарий к продаваемому автомобилю. В нем присутствует мусор, от которого нужно будет избавиться. План действий будет примерно следующим:
* Очистим строки от ненужных символов
* Проведем лемматизацию строк
* Уберем стоп-слова из строк
* Проведем токенизацию текста
* Проведем векторизацию текста

## 6.1. Лемматизация

Суть лемматизации заключается в том, чтобы вернуть каждое слово к его изначальной форме. Это нужно, чтобы не считать разные формы слова "уникальными" при обработке текста.

Загрузим морфологический анализатор и скопируем данные из исходного датасета:

In [ ]:
morphy = pymorphy2.MorphAnalyzer()
df_NLP = data.copy()

Создадим паттерн с "мусорными" символами в строках и напишем функцию для лемматизации:

In [ ]:
trash_sym = "[^А-я ]+" #Более точная очистка, которая оставляет только слова

In [ ]:
# Паттерн с символами
#trash_sym = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-–»«•∙·✔➥●☛“”°№₽®]+"

# функция для лемматизации текста:


def lemma(text):
    text = text.lower()  # понижаем регистр
    text = re.sub(trash_sym, ' ', text)  # удаляем символы из паттерна
    strings = []  # создаем массив, в котором будут храниться лемматизированные строки
    for wrd in text.split():  # берем слово из строки
        wrd = wrd.strip()  # убираем пробелы до и после слова
        wrd = morphy.normal_forms(wrd)[0]  # приводим к нормальной форме
        strings.append(wrd)  # добавляем слово в строку массива
    return ' '.join(strings)  # вернем значения, разделив пробелами

Применим функцию к нашему nlp-датасету:

In [ ]:
strings_set = []
strings_set = df_NLP.apply(
    lambda df_NLP: lemma(df_NLP.description), axis=1)

Загрузим список русских стоп-слов:

In [ ]:
russian_stopwords = stopwords.words("russian")

Напишем функцию, которая которая будет проверять наши лемматизированные слова на наличие в списке стоп-слов:

In [ ]:
# функция для проверки на стоп-слова
def lineWithoutStopWords(line):
    line = line.split()  # разделяем на слова
    # возвращаем слово, если оно не в списке стоп-слов
    return [word for word in line if word not in russian_stopwords]


# применим функцию к нашим лемматизированым строкам слов
str_without_stop = [lineWithoutStopWords(line) for line in strings_set]

Укажем обучающую\валидационную и тестовую выборки:

In [ ]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

## 6.2. Токенизация

Пропишем параметры для токенизатора, с помощью которого будем проводить токенизацию текста:

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

Обучим токенизатор на полученном списке лемматизированых слов:

In [ ]:
%%time
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(str_without_stop)

Проверим результат:

In [ ]:
tokenize.word_index

Используем полученный список слов для векторизации текста из наших выборок:

In [ ]:
%%time
text_train_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_train), maxlen=MAX_SEQUENCE_LENGTH)
text_test_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_test), maxlen=MAX_SEQUENCE_LENGTH)
text_sub_sequences = sequence.pad_sequences(
    tokenize.texts_to_sequences(text_sub), maxlen=MAX_SEQUENCE_LENGTH)

print(text_train_sequences.shape,
      text_test_sequences.shape, text_sub_sequences.shape, )

Выведем пример того, как теперь выглядит текст:

In [ ]:
print(text_train.iloc[6])
print(text_train_sequences[6])

## 6.3. RNN NLP

In [ ]:
model_nlp = Sequential()
model_nlp.add(L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"))
model_nlp.add(L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,))
model_nlp.add(L.LayerNormalization())
model_nlp.add(L.LSTM(256, return_sequences=True))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.Dense(128, activation="sigmoid"))
model_nlp.add(L.Dropout(0.5))
model_nlp.add(L.LSTM(64,))
model_nlp.add(L.Dropout(0.25))
model_nlp.add(L.Dense(64, activation="relu"))
model_nlp.add(L.Dropout(0.25))

После создания NLP-сети мы переходим к сети для табличной обработки данных

## 6.4. MLP-сеть

In [ ]:
model_mlp = Sequential()
model_mlp.add(L.Dense(512, input_dim=X_train.shape[1], activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(256, kernel_regularizer=regularizers.l2(
    l2=1e-6), activation="relu"))
model_mlp.add(L.Dropout(0.5))
model_mlp.add(L.Dense(128, kernel_regularizer=regularizers.l2(
    l2=1e-5), activation="relu"))
model_mlp.add(L.Dropout(0.25))

Теперь, когда обе сети готовы, создадим "голову", которая будет объединять выходы наших сетей воедино.

## 6.5. Multiple Inputs NN

На этом этапе было рассмотрено несколько вариантов с различными функциями активации и количеством юнитов. Как и ранее, финальный вариант, который дал наиболее низкую MAPE, представлен ниже:

In [ ]:
combinedInput = L.concatenate([model_nlp.output, model_mlp.output])
# being our regression head
head = L.Dense(32, activation="relu")(combinedInput)
#head_1 = L.Dense(16, activation="relu")(head)
head_2 = L.Dense(1, activation="linear")(head)

model = Model(inputs=[model_nlp.input, model_mlp.input], outputs=head_2)

Выведем описание модели:

In [ ]:
model.summary()

## 6.6. Обучение

Adam в данном случае сходится гораздо быстрее и дает более стабильные значения исследуемой метрики. Начальную скорость обучения выберем 0.01. 

Проведем компиляцию модели:

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
# Проведем компиляцию модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

В процессе исследования, было выявлено, что фиксированная скорость обучения дает не самый хороший результат. Поэтому, помимо уже имеющихся коллбэков, которые мы упоминали ранее при обучении табличной сети, добавили ReduceLROnPlateau, который уменьшает скорость обучения, в случае когда нет улучшения исследуемой метрики в течение **patience** эпох. 

Объявим коллбэки:

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop = EarlyStopping(
    monitor='val_MAPE', patience=15, restore_best_weights=True,)
lr_scheduler = ReduceLROnPlateau(monitor='val_MAPE',
                                 factor=0.5,  # уменьшим lr в 2 раза
                                 patience=5,  # если нет улучшения через 5 эпохи - уменьшить lr
                                 min_lr=0.0001,  # минимальная скорость обучения
                                 verbose=1,  # выводить сообщения об уменьшении скорости
                                 mode='auto')  # выбранный способ отслеживания метрики
callbacks_list = [checkpoint, earlystop, lr_scheduler]

Теперь приступим к обучению модели. Оно проходит довольно быстро. Фактически, мы все равно сталкиваемся с тем, что процесс обучения прерывается с помощью EarlyStopping. Но с использованием ReduceLROnPlateau точность удалось несколько повысить по сравнению с предыдущими результатами исследований.

Запустим обучение модели:

In [ ]:
# Запустим обучение модели
history = model.fit([text_train_sequences, X_train], y_train,
                    batch_size=512,
                    epochs=500,
                    validation_data=([text_test_sequences, X_test], y_test),
                    callbacks=callbacks_list
                    )

Когда обучение завершилось, посмотрим на график нашей функции потерь:

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

На определенном промежутке обучения график ведет себя "неспокойно". Это может быть связано с тем, что в начале мы используем довольно большую скорость обучения, быстро спускаемся в "плато", и пока скорость обучения не уменьшится, возникают те самые скачки функции потерь.

Сохраним лучшие веса модели:

In [ ]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_mlp_nlp.hdf5')

Посмотрим значение полученной метрики:

In [ ]:
test_predict_nn2 = model.predict([text_test_sequences, X_test])
print(f"TEST mape: {(mape(y_test, test_predict_nn2[:,0]))*100:0.2f}%")

До правки архитектуре в голове метрика была на процент хуже - после правки мы получили вот такой вот результат - 10,63 против 10,78, что вполне неплохо (для чистоты эксперимента нужно модифицировать верх и в первой сети, и посмотреть, что изменится).

In [ ]:
sub_predict_nn2 = model.predict([text_sub_sequences, X_sub])
sample_submission['price'] = sub_predict_nn2[:, 0]
sample_submission.to_csv('nn2_submission.csv', index=False)

На данном этапе исследования было достигнуто значение MAPE=11.97%. Не самый хороший результат: получилось сильно хуже, чем смесь ML и табличной нейросети. (До апдейта архитектуры).

После апдейта не будем терять наши ценные сабмиты, и сразу выполним смешение трех систем. Коэффициенты для смешения будем искать через минимизацию MAPE симплекс-методом:


In [ ]:
N_models = 3
initial_guess = np.ones(N_models-1)/(N_models)
bnd = ((-1,1),(-1,1), (-1,1))
predictions = [test_predict_catboost,test_predict_nn1[:,0], test_predict_nn2[:,0]]
fun = lambda x: mape_function(x,predictions)
res = minimize(fun, initial_guess, method='Nelder-Mead', tol=1e-6)
res.x

In [ ]:
final_pred_2 = (1-np.sum(res.x))*predictions[0]
for i in range(N_models-1):
    final_pred_2 += res.x[i]*predictions[i+1]
mape(y_test, final_pred_2)

Сравним значения:

In [ ]:
mape(y_test, final_pred)

Выведем на сабмит уже вот именно этот результат:

In [ ]:
optimized_sub_pred_2 = (1-np.sum(res.x))*sub_predict_catboost
optimized_sub_pred_2 += res.x[0]*sub_predict_nn1[:, 0]
optimized_sub_pred_2 += res.x[1]*sub_predict_nn2[:, 0]

sample_submission['price'] = optimized_sub_pred_2
sample_submission.to_csv('blend_submission_optimized_N3.csv', index=False)

Итог на сабмите - 10.86 против 10.77 у модели без NLP. - Наблюдается понижение метрики. Возможно, виновата недостаточная очистка датасета?
Можно идти четырьмя путями:
1. Изменить архитектуру первой сети, добавив ещё один слой (возможно, система переобучается, что приводит к проседанию MAPE на валидации) (Тогда, если счет просядет и там, это будет уже заметно) Результат: Поднятие MAPE на Модели ML+NN до 10.68.
2. Изменить архитектуру второй сети, убрав дополнительный слой, и попробовав смешать её предсказания с остальными моделями.
3. Прикрутить Image Processing 

# 7. Model 5: NLP+MLP+EffNetB6 Multi Input NN

На данном этапе, к multi-input сети, которую мы исследовали п.6. добавляется нейронная сеть, которая занимается обработкой изображений.

## 7.1. Image Data

Загрузим изображения автомобилей, которые будет обрабатывать наша сеть. Для начала проверим, что все имеющиеся у нас в input'е данные подгружены корректно, выведем 9 изображений из папки img, используя для этого sell_id, как название изображения:

In [ ]:
# Укажем размер выводимого изображения
plt.figure(figsize=(10, 6))
# 9 случайных примеров из train
random_image = train.sample(n=9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values
# выведем 9 изображений автомобилей и цен к ним
for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Как видно, все вывелось корректно, никаких нареканий не имеется. Теперь загрузим все имеющиеся изображения с форматом 320х240:

In [ ]:
# Установим размер изображения
size = (320, 240)
# функция для загрузки изображений


def get_image_array(index):
    images_set = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)  # изменение размера
        images_set.append(image)  # добавляем изображение в массив
    images_set = np.array(images_set)
    print('images shape', images_set.shape, 'dtype', images_set.dtype)
    return(images_set)


# применим функцию для создания выборок
images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

## 7.2. Настройки albumentations

In [ ]:
IMG_SIZE = [size[1],size[0]]

In [ ]:
AUGMENTATIONS_TRAIN = A.Compose([ #Для тренировки нейросети
    A.GaussianBlur(p=0.05),
    A.ShiftScaleRotate(shift_limit=0.08, 
                       scale_limit=0.05, 
                       interpolation=1, 
                       border_mode=4, 
                       rotate_limit=15, 
                       p=.5),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, 
                                                contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, 
                                                contrast_limit=0.1)],
        p=0.5),
    A.HorizontalFlip(p=0.4),
 #   A.HueSaturationValue(p=0.4),
    A.RGBShift(p=0.4),
    A.FancyPCA(alpha=0.1, 
               always_apply=False, p=0.4),
    A.Resize(IMG_SIZE[0],IMG_SIZE[1]),
  #  A.ToFloat(max_value = 255)
])
AUGMENTATIONS_TEST = A.Compose([    #Аугментации разного уровня сложности для сетей - некоторый чуть упрощенный вариант, просто ресайз, и более сложные аугментации с вырезами
    A.ShiftScaleRotate(shift_limit=0.04, 
                       scale_limit=0.025, 
                       interpolation=1, 
                       border_mode=4, 
                       rotate_limit=10, 
                       p=.75),
    A.HorizontalFlip(p=0.5),
    A.Resize(IMG_SIZE[0],IMG_SIZE[1]),
   # A.ToFloat(max_value = 255),
    ])

AUGMENTATIONS_TEST_CLEAR = A.Compose([
#    A.ToFloat(max_value = 255)
    ])

AUGMENTATIONS_TEST_HARD = A.Compose([   #Слишком серьезные аугментации, под них нужно переучивать модель специально
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
    A.OneOf([
        A.CenterCrop(height=220, width=200),
        A.CenterCrop(height=200, width=220),
    ],p=0.5),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=0.5),
    A.GaussianBlur(p=0.05),
    A.HueSaturationValue(p=0.5),
    A.RGBShift(p=0.5),
    A.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    A.Resize(IMG_SIZE[0],IMG_SIZE[1]),
#    A.ToFloat(max_value = 255)
])

# Выведем пример аугментации
plt.figure(figsize=(12, 8))
for i in range(9):
    img = AUGMENTATIONS_TRAIN(image=images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

Выше показан пример аугментации для имеющихся изображений.

 Для обработки данных при помощи tf.data.Dataset необходимо провести некоторую подготовку данных. Начнем с обучения нашего токенизатора:

In [ ]:
# обучение токенизатора для NLP
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(str_without_stop)

Теперь создадим функции, которые будут применяться к имеющимся у нас данным:

In [ ]:
# аугментация изображений
def process_image(image):
    return AUGMENTATIONS_TRAIN(image=image.numpy())['image']
# векторизация строки


def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen=MAX_SEQUENCE_LENGTH)
# применение векторизации к тексту


def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]
# функция  для применения вышеупомянутых функций к входным данным обучающей выборки


def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image, ] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text, ] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price
# функция  для применения вышеупомянутых функций к входным данным валидационной и тестовой выборкок


def tf_process_val_dataset_element(image, table_data, text, price):
    [text, ] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price


# использование tf.data.Dataset с использованием функций для обучающей выборки
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
)).map(tf_process_train_dataset_element)
# использование tf.data.Dataset с использованием функций для валидационной выборки
test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
)).map(tf_process_val_dataset_element)
# использование tf.data.Dataset с использованием функций для тестовой выборки
y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
)).map(tf_process_val_dataset_element)

# проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__()
test_dataset.__iter__().__next__()
sub_dataset.__iter__().__next__()

## 7.4. Fine-tune EfficientNet

Рассмотрим для начала прирост эффективности при использования ENetB3. Более сложные сети обучим только в том случае, если прирост будет наблюдаться.

Загрузим исследуемую модель:

In [ ]:
# загрузим модель без "головы" и укажем, что она может обучаться
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(
    weights='imagenet', include_top=False, input_shape=(size[1], size[0], 3))


In [ ]:
efficientnet_model.trainable = False

Добавим output-слой для нашей EfficienNet модели:

In [ ]:
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

Архитектура Табличной нейронной сети и RNN NLP взята из п.6.

In [ ]:
# Табличная нейронная сеть
tabular_model = Sequential([
    L.Input(shape=X.shape[1]),
    L.Dense(512, input_dim=X_train.shape[1], activation="relu"),
    L.Dropout(0.5),
    L.Dense(256, kernel_regularizer=regularizers.l2(
        l2=1e-6), activation="relu"),
    L.Dropout(0.5),
    L.Dense(128, kernel_regularizer=regularizers.l2(
        l2=1e-5), activation="relu"),
    L.Dropout(0.25)
])

In [ ]:
# Нейронная сеть для NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LayerNormalization(),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.Dense(128, activation="sigmoid"),
    L.Dropout(0.5),
    L.LSTM(64,),
    L.Dropout(0.25),
    L.Dense(64, activation="relu"),
    L.Dropout(0.25),
])

Теперь, когда мы создали все три нейронных сети, объединим их с помощью единой "головы". Здесь было исследовано 2 варианта головы с различным количеством юнитов в dense-слое (256 и 128), финальный вариант представлен ниже:

In [ ]:
# объединяем выходы трех нейросетей
combinedInput = L.concatenate(
    [efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(128, activation="relu")(combinedInput)


head_1 = L.Dense(64, activation="relu")(head)


head_2 = L.Dense(1,)(head_1)
# Соберем наши части в одну модель
model = Model(inputs=[efficientnet_model.input,
                      tabular_model.input, nlp_model.input], outputs=head_2)
# посмотрим описание нашей модели
model.summary()

Теперь можно приступать к компиляции и обучению нашей модели:

In [ ]:
# укажем используемый оптимизатор и начальную скорость обучения
optimizer = tf.keras.optimizers.Adam(2e-3)
# компиляция модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

Немного изменим список наших коллбэков:
1. Для EarlyStopping укажем 10 эпох для завершения обучения
2. Для ReduceLROnPlateau:
    * укажем 3 эпохи для изменения скорости обучения
    * установим минимальную скорость обучения 0.00001

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop1 = EarlyStopping(
    monitor='val_MAPE', patience=10, restore_best_weights=True,)
lr_scheduler1 = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.5,  # уменьшим lr в 2 раза
                                  patience=3,  # если нет улучшения через 3 эпохи - уменьшить lr
                                  min_lr=0.00001,  # минимальная скорость обучения
                                  verbose=1,  # выводить сообщения об уменьшении скорости
                                  mode='auto')  # выбранный способ отслеживания метрики
callbacks_list1 = [checkpoint, earlystop1, lr_scheduler1]

Теперь обучим нашу модель:

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=45,
                    validation_data=test_dataset.batch(30),
                    callbacks=callbacks_list1
                    )

Посмотрим, как выглядит функция потерь:

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

Как можно увидеть, в начале функция потерь валидационной выборки ведет себя нестабильно, то растет, то падает. Однако, при уменьшении скорости обучения, она становится более гладкой, резких скачков уже не наблюдается.

Загрузим лучшие веса модели и сохраним модель. Подготовим разморозку 1/2 весов ENetB3:

In [ ]:
model.load_weights('../working/best_model.hdf5')
#model.load_weights('../working/nn_final.hdf5')
model.save('../working/nn_final.hdf5')

In [ ]:
efficientnet_model.trainable = True
nlp_model.trainable = True
tabular_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = round(len(efficientnet_model.layers)*2/4)

# Freeze all the layers before the `fine_tune_at` layer
for layer in efficientnet_model.layers[:fine_tune_at]:
    layer.trainable =  False

In [ ]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop1 = EarlyStopping(
    monitor='val_MAPE', patience=5, restore_best_weights=True,)
lr_scheduler1 = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.5,  # уменьшим lr в 2 раза
                                  patience=2,  # если нет улучшения через 3 эпохи - уменьшить lr
                                  min_lr=1e-7,  # минимальная скорость обучения
                                  verbose=1,  # выводить сообщения об уменьшении скорости
                                  mode='auto')  # выбранный способ отслеживания метрики
callbacks_list1 = [checkpoint, earlystop1, lr_scheduler1]

In [ ]:
# укажем используемый оптимизатор и начальную скорость обучения
optimizer = tf.keras.optimizers.Adam(1e-5)
# компиляция модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_dataset.batch(30),
                    epochs=12,
                    validation_data=test_dataset.batch(30),
                    callbacks=callbacks_list1
                    )

In [ ]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

Посмотрим, как себя будет вести MAPE:

In [ ]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

На валидационных данных мы получили метрику немного хуже, чем у простой табличной сети, но лучше чем у multi-input сети без обработки изображений. Это говорит нам о том, что работа проделана не зря и положительный результат наблюдается. Сделаем сабмит:

In [ ]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn3[:, 0]
sample_submission.to_csv('nn3_submission.csv', index=False)

На тестовых данных метрика составила около 10.4-10.5 в зависимости от длительности обучения. В общем-то неплохо! Попробуем смешать:

# 8. Blending и Final Submission

В качестве конечного результата используем комбинацию из результатов предсказаний некоторых предыдущих моделей:
1. CatBoostRegressor
2. Табличная нейронная сеть
3. NLP+MLP+EffNetB3Multi Input NN

In [ ]:
N_models = 4
initial_guess = np.ones(N_models-1)/(N_models)
predictions = [test_predict_catboost,test_predict_nn1[:,0], test_predict_nn2[:,0],test_predict_nn3[:,0]]
fun = lambda x: mape_function(x,predictions)
res = minimize(fun, initial_guess, method='Nelder-Mead', tol=1e-6)
res.x

In [ ]:
final_pred_3 = (1-np.sum(res.x))*predictions[0]
for i in range(N_models-1):
    final_pred_3 += res.x[i]*predictions[i+1]
mape(y_test, final_pred_3)

Мы добились самой хорошей метрики, которая у нас была! На этом этапе сделаем финальный сабмит:

In [ ]:
optimized_sub_pred_final = (1-np.sum(res.x))*sub_predict_catboost
optimized_sub_pred_final += res.x[0]*sub_predict_nn1[:, 0]
optimized_sub_pred_final += res.x[1]*sub_predict_nn2[:, 0]
optimized_sub_pred_final += res.x[2]*sub_predict_nn3[:, 0]

sample_submission['price'] = optimized_sub_pred_final
sample_submission.to_csv('blend_submission_optimized_N4.csv', index=False)

Мы получили результат MAPE=10.7%! Данный результат оказался немного хуже простого смешения ML и табличной нейросети. Видимо, нужно изменять архитектуру сети или сам способ обработки информации, потому что сеть плохо понимает, как признаки, извлекаемые из текста и изображения, влияют на итоговую стоимость. Возможно, для понимания влияния на изображение она просто недостаточно хороша и сложна, а для текста она недостаточно хорошо понимает связь слов в предложении, так как использовалась простая рекуррентная нейросеть, поэтому мы наблюдаем некоторое недопонимание, выражающееся в снижении MAPE.

# 9. Выводы

По результатам выполнения  проекта были получены следующие метрики MAPE:
1. MLP + NLP + EffNetB3 - не проверялась отдельно
2. MLP + RNN NLP  - 11.97%
3. CatBoostRegressor - 11.75%
4. Smiple Dense NN - 11.03%
5. Blending(все 4 модели с оптимизацией коэффициентов по Нелдеру-Миду) - 10.7%
6. Blending(по трем моделям: CatBoost + табличная нейросеть + RNN NLP) - 10.69%
7. Blending(две модели: CatBoost + табличная нейрость) - 10.68%

По итогу, получили неплохой по качеству результат. 
